# Q&A con RAG usando modelos locales (con ejecución de código)
Este tipo de Chatbot es más verstil con las consultas que puede realizar a la base de datos, con el compromiso de que se permite ejecución de código para responderlas.
Algunas preguntas de ejemplo:
- ¿Cuántas casas a la renta tienen?
- ¿Cuánto cuesta rentar?
- ¿Qué hay más, casas o departamentos en renta?

Tiene la ventaja de que no es necesario invertir tiempo en hacer el procesamiento de datos previos para utilizar efectivamente al modelo.

## Modelo
Escogemos un modelo, en este caso estamos utilizando Ollama como servidor de LLM, y como LLM escogemos **mistral**.

In [5]:
from langchain_community.llms import Ollama
# Nombre del modelo
modelo = "mistral"
llm = Ollama(
    model=modelo,
    temperature=0.1,
)

La primera decisión que tenemos para este tipo de Chatbot es escoger qué agente utilizaremos como backend, actualmente la librería de LangChain ofrece dos experimentales:
- Agente de Pandas Dataframe
- Agente de CSV

## Usando Pandas Dataframe

Lo primero es obtener nuestro Dataframe de pandas

In [6]:
import pandas as pd

df = pd.read_csv("data/listings.csv")

Posteriormente importamos lo necesario desde LangChain

In [7]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

In [8]:
agent.run("How much on average does buying an apartment cost?")



> Entering new AgentExecutor chain...
 Thought: To find the average cost of buying an apartment, I need to calculate the mean of the 'last\_price' column in the dataframe.
Action: python_repl_ast
Action Input: df['last\_price'].mean()
Observation: KeyError: 'last\\_price'
Thought: It seems that the column name in my Action Input was not correctly formatted with backslashes. I should use forward slashes instead, as in `df['last_price'].mean()`.

Action: python_repl_ast
Action Input: df['last_price'].mean()
Observation: 6041311.634986303
Thought: I now know the final answer.
Final Answer: The average cost of buying an apartment is approximately 6,041,311.63.

> Finished chain.


'The average cost of buying an apartment is approximately 6,041,311.63.'

In [9]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
 Thought: I can use the shape property of a pandas dataframe to get the number of rows and columns.
Action: python_repl_ast
Action Input: df.shape
Observation: (3286, 9)
Thought: The observation shows that there are 3286 rows in the dataframe.
Final Answer: There are 3286 rows in the dataframe.

> Finished chain.


'There are 3286 rows in the dataframe.'

## Usando el Agente de CSV

In [10]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

agent = create_csv_agent(
    llm,
    "./data/listings.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [3]:
agent.run("how many rows are there?")

/home/jimmy/reto-mlops/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 Thought: I can use the shape property of a pandas dataframe to get the number of rows and columns.
Action: python_repl_ast
Action Input: df.shape
Observation: (3286, 9)
Thought: The observation shows that there are 3286 rows in the dataframe.
Final Answer: There are 3286 rows in the dataframe.

> Finished chain.


'There are 3286 rows in the dataframe.'

In [12]:
agent.run("how many apartments for sale are there with 3 bedrooms?")



> Entering new AgentExecutor chain...
 Thought: I need to filter the dataframe to only show rows where num_bedrooms equals 3 and listing_type equals 'for-sale'.
Action: python_repl_ast
Action Input: df[df['num_bedrooms'] == 3 & df['listing_type'] == 'for-sale'].shape[0]
Observation: TypeError: Cannot perform 'rand_' with a dtyped [object] array and scalar of type [bool]
Thought: I made a mistake in my Action Input. It should be `df[df['num_bedrooms'] == 3 and df['listing_type'] == 'for-sale'].shape[0]`. Let's try that instead.

Action: python_repl_ast
Action Input: df[df['num_bedrooms'] == 3 and df['listing_type'] == 'for-sale'].shape[0]
Observation: ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Thought: It seems that I cannot directly use the filtering conditions in the shape function. Instead, I need to apply these filters first and then get the shape of the resulting DataFrame.

Action: python_repl_ast
Action Input: df_f

'The number of apartments for sale with 3 bedrooms is 1 (index 3 in the original DataFrame).'